In [2]:
#%pip install transformers
#%pip install datasets
import pandas as pd
import transformers
from transformers import TFBartForConditionalGeneration, BartTokenizer, BartConfig, TFGPT2LMHeadModel,GPT2Tokenizer,GPT2Config, AutoTokenizer,AutoConfig
from sklearn.model_selection import train_test_split
from tensorflow.data import Dataset
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model, save_model, load_model
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import re
import os
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Add,Activation
from tensorflow.keras.models import Sequential
from tensorflow.random import set_seed


In [3]:
# Read the csv data
#sample_English = pd.read_csv('/content/gdrive/MyDrive/sample_English_lyrics_10000.csv')
#sample_English.head()

new_data = pd.read_csv('./lyrics_training_data.csv')

# Explore the top 5 rows of the dataset
new_data.head()

,Unnamed: 0.1,Unnamed: 0,link,artist,song_name,lyrics
0,15191,15191,../lyrics/michaellearnstorock/outoftheblue.html,Michael Learns To Rock Lyrics,Out Of The Blue,\n\r\nI was almost about to lose my faith\r\nW...
1,17643,17643,../lyrics/beatles/illbeonmyway.html,The Beatles Lyrics,I'll Be On My Way,\n\r\nThe sun is fading away\nThat's the end o...
2,5081,5081,../lyrics/johnnycash/idjustbefoolenoughtofall....,Johnny Cash Lyrics,I'd Just Be Fool Enough (To Fall),\n\r\nOh please don't be so careless with your...
3,21442,21442,https://www.azlyrics.com/lyrics/nerd/1000.html,N.E.R.D & Future Lyrics,1000,"\n\n[Pharrell Williams:]\nAh, ah\nAssembling a..."
4,10461,10461,../lyrics/snoopdogg/wasntyourfault.html,Snoop Dogg Lyrics,Wasn't Your Fault,\n\n[Voice talking (echo)]\r\nYeah .. niggas w...


In [5]:
# download and initialize the pre-trained tokenizer and model
def load_pretrained():
    # tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    # config = GPT2Config.from_pretrained("gpt2")
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    #tokenizer = AutoTokenizer.from_pretrained('gpt2', is_split_into_words=True)
    config = GPT2Config.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id, use_cache=False)
    model = TFGPT2LMHeadModel.from_pretrained("gpt2")#, config = config)
    return tokenizer, config, model

tokenizer, config, model= load_pretrained()
#model = load_model(tokenizer.eos_token_id)
#model = TFGPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)
#model = TFGPT2LMHeadModel.from_pretrained("gpt2", config = config)
#model.resize_token_embeddings(len(tokenizer))

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [6]:
# Put the lyrics as string into one list
#Lyrics = [text for idx, text in enumerate(sample_English['Lyric'])]
Lyrics = [text for text in new_data['lyrics']]
Lyrics[:3]

["\n\r\nI was almost about to lose my faith\r\nWas still dreaming but feared it was too late\n\r\nBut then you came along to my surprise\r\nAnd stole my heart before my very eyes\n\n[Chorus:]\r\nYou took me right out of the blue\r\nSimply by showing that you love me too\r\nOnly by giving me your everything\r\nWith a love so true you took me out of the blue\n\r\nI was wondering what love was all about\r\nI was trying but couldn't work it out\n\r\nBut then you came along to my surprise\r\nAnd made my frozen mind come alive\n\n[Chorus:]\r\nYou took me right out of the blue\r\nSimply by showing that you love me too...\n\r\nYou let me out of the darkness \r\nYou brought me out in the sun\r\nI think you must be the only one for me\r\n'cos you took me\n\n[Chorus:]\r\nRight out of the blue\r\nSimply by showing that you love me too\r\nOnly by giving me your everything\r\nBreathing air below my wings\r\nYou took me right out of the night\r\nSimply by filling my heart with light\r\nOnly by giving

In [7]:
#clean and preprocess the lyrics
def replace_char(lyrics):
#     for i in range(len(lyrics)):
#         lyrics[i] = re.sub("[\(\[].*?[\)\]]",'',lyrics[i])
#         lyrics[i] = re.sub(r'\.+', ".", lyrics[i])
#         lyrics[i] = lyrics[i].replace("\r", "").replace("\n", " ").replace(',', '').replace("\'",'').replace('/','').replace('-','').replace('...)','')
#     return lyrics
        """
        - remove any html tags (< /br> often found)
        - Keep only ASCII + Latin chars, digits and whitespaces
        - pad punctuation chars with whitespace
        - convert all whitespaces (tabs etc.) to single wspace
        """
        for i in range(len(lyrics)):
            RE_PUNCTUATION = re.compile("([!?.,;-])")
            #RE_TAGS = re.compile(r"<[^>]+>")
            RE_TAGS = re.compile(r"[\(\[].*?[\)\]]")
            RE_ASCII = re.compile(r"[^A-Za-z,.!?0-9 ]", re.IGNORECASE)
            RE_WSPACE = re.compile(r"\s+", re.IGNORECASE)
            lyrics[i] = re.sub(RE_TAGS, " ", lyrics[i])
            lyrics[i] = re.sub(RE_ASCII, " ", lyrics[i])
            lyrics[i] = re.sub(RE_PUNCTUATION, r" ", lyrics[i])
            lyrics[i] = re.sub(RE_WSPACE, " ", lyrics[i])
            lyrics[i] = lyrics[i].lower()
        return lyrics

    
Lyrics_new = replace_char(Lyrics)

In [8]:
# add eos tokens to the end of the text
Lyrics_new = [i + tokenizer.eos_token for i in Lyrics]
Lyrics_new[:3]

[' i was almost about to lose my faith was still dreaming but feared it was too late but then you came along to my surprise and stole my heart before my very eyes you took me right out of the blue simply by showing that you love me too only by giving me your everything with a love so true you took me out of the blue i was wondering what love was all about i was trying but couldn t work it out but then you came along to my surprise and made my frozen mind come alive you took me right out of the blue simply by showing that you love me too you let me out of the darkness you brought me out in the sun i think you must be the only one for me cos you took me right out of the blue simply by showing that you love me too only by giving me your everything breathing air below my wings you took me right out of the night simply by filling my heart with light only by giving me your energy with a love so true you took me out of the blue <|endoftext|>',
 ' the sun is fading away that s the end of the d

In [9]:
#split train & test data
train_data, test_data = train_test_split(Lyrics_new, test_size = .3, random_state=42)

train_data, val_data = train_test_split(train_data, test_size = .1, random_state=42)

In [10]:
#split predictor and labels
def shift_right(data):
    words = []
    predictor_word =[]
    labels_word = []
    for i in range(len(data)):
        words.append(list(filter(lambda x: x !='', re.split(r' ', data[i]))))
        predictor_word.append(words[i][:-1])
        labels_word.append(words[i][1:])
        
        
    
    predictor_list =[]
    labels_list = []
    for list_word in predictor_word:
        predictor_list.append(' '.join(list_word))
    for list_word in labels_word:
        labels_list.append(' '.join(list_word))

    return predictor_list, labels_list



predictor_train, labels_train= shift_right(train_data)

predictor_val, labels_val =  shift_right(val_data)

In [11]:
#tokenize the inputs for training and validating datset

max_len = 180

tokenizer.pad_token = tokenizer.eos_token 

predictor_train_token = tokenizer(predictor_train, return_tensors= 'tf',truncation = True, max_length = max_len, padding='max_length')
# labels_train = tokenizer(labels_train, return_tensors= 'tf',truncation = True, max_length = max_len, padding='max_length')
# labels_train_token = tokenizer(labels_train,truncation = True, max_length = max_len, padding='max_length')

predictor_val_token = tokenizer(predictor_val,return_tensors= 'tf',truncation = True, max_length = max_len, padding='max_length')
#labels_val = tokenizer(labels_val, return_tensors= 'tf',truncation = True, max_length = max_len, padding='max_length')

labels_train_token = tokenizer(labels_train,truncation = True, max_length = max_len, padding='max_length')
labels_val_token = tokenizer(labels_val, truncation = True, max_length = max_len, padding='max_length')

In [12]:
# set label's padding token labels to -100 which is ignored in loss computation
def ignore_label_pad_token(labels_input_ids):
    for idx, i in enumerate(labels_input_ids):
        for t in range(len(i)):
            if i[t]==tokenizer.eos_token_id:
                i[t] = -100
            else:
                i[t] = i[t]
                
    return labels_input_ids

labels_train_new = ignore_label_pad_token(labels_train_token.input_ids)
labels_val_new = ignore_label_pad_token(labels_val_token.input_ids)

In [13]:
#convert arrays to tensors
labels_train_input_ids = tf.convert_to_tensor(labels_train_new, dtype=tf.int32)
labels_val_input_ids = tf.convert_to_tensor(labels_val_new, dtype=tf.int32)

In [14]:
#convert labels which are array to tensors;
#create tensorflow dataset that include all the inputs and outputs for model.fit, 
#shuffle, batch and prefech the dataset

train_dataset = Dataset.from_tensor_slices(({'input_ids': predictor_train_token['input_ids'], 'attention_mask':predictor_train_token['attention_mask']}, labels_train_input_ids))
train_dataset = train_dataset.shuffle(buffer_size =len(predictor_train_token),reshuffle_each_iteration=True)
train_dataset = train_dataset.batch(8)
train_dataset = train_dataset.prefetch(5)

val_dataset = Dataset.from_tensor_slices(({'input_ids':predictor_val_token['input_ids'],'attention_mask':predictor_val_token['attention_mask']}, labels_val_input_ids))
val_dataset = val_dataset.shuffle(buffer_size =len(predictor_val_token),reshuffle_each_iteration=True)
val_dataset = val_dataset.batch(8)
val_dataset = val_dataset.prefetch(5)


In [15]:
# set customized learning rate, compile model

model.resize_token_embeddings(len(tokenizer))
lr_schedule = optimizers.schedules.PolynomialDecay(
initial_learning_rate=1e-5,
decay_steps=10000,
end_learning_rate=0, 
power=1.0)
# lr_schedule = optimizers.schedules.ExponentialDecay(
# initial_learning_rate=1e-6,
# decay_steps=1000,
# decay_rate=0.7,
# staircase=True)


model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    #loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    loss = model.compute_loss,
    metrics=tf.keras.metrics.SparseCategoricalAccuracy())

In [ ]:
#create checkpoint 
folder = '/content/lyrics_gpt'
if not os.path.exists(folder):
    os.makedirs(folder)
#filepath = '/Users/mycelebs_129/model6-{epoch:02d}-{val_sparse_categorical_accuracy:.2f}.hdf5'
filepath = os.path.join(folder,  'lyrics_gen_BOS')

checkpoint_callback = ModelCheckpoint(filepath, monitor='val_loss', mode = 'min', save_freq='epoch',save_best_only=True, save_weights_only=False)


In [ ]:
# train model and save best model 
model.fit(train_dataset,
            validation_data = val_dataset,
            epochs =1,
            callbacks = [checkpoint_callback]
            )

In [19]:
# load the pretrained model weights
model.load_weights("./lyrics_gen_model")


<tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[12254,   502,   284,   262,  8824]], dtype=int32)>

In [40]:
#first sentence of your lyrics
first_sen = "look up in the sky"
first_sen_input = tokenizer(first_sen, return_tensors = 'tf').input_ids
first_sen_input


<tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[5460,  510,  287,  262, 6766]], dtype=int32)>

In [44]:
#Generate lyrics 
#can play with the parameters to generate most ideal results
set_seed(42)
result = model.generate(first_sen_input, 
                        do_sample=True,
                        #length_penalty =1.5,
                        repetition_penalty=1.4,
                        max_length=200,
                        top_p=0.90, 
                        #top_k=50,
                        #early_stopping = False,
                        temperature =0.8,
                        #num_beams = 20,
                        #num_return_sequences =5
                        #truncate="<|endoftext|>",
                        #result_as_list = True
                        )

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


In [45]:
#lyrics_gen = [tokenizer.decode(i, skip_special_tokens = True) for i in result]
lyrics_gen = tokenizer.decode(result[0], skip_special_tokens = True)

In [46]:
lyrics_gen

'look up in the sky and see what we can do We re here to give you a ride Let us take that dream away from your mind Take it all back out of our heads Oh no You ve got me down by my side oh yeah What happened s wrong I m taking this trip And if ya want some more than just another meal then let yo ll be alright Come on baby cause they say come tryin for once They don t know how far apart There are so many different ways there is nothing left To get close enough Thats why when he finally cries his tears fall short but not long Before sunrise He gets better When those little things begin falling But only until one day will wake him Up again with every step Back home alone Is everything bad Baby It seems like life ain still going well So now stop crying cry Just start feeling good Now love won never break Cause nobody ever knows who died Ain d been waiting Don T worry darling My girl has found something sweet If she wants anything else Then please show'